In [1]:
import sys
print(sys.executable)

/home/is/lathifgalih-k/miniconda3/envs/myenv/bin/python3.10


In [2]:
import os
os.getcwd()

'/home/is/lathifgalih-k/research_naist/multimodal-clone/ipynb'

In [3]:
%cd ../vits

/home/is/lathifgalih-k/research_naist/multimodal-clone/vits


/home/is/lathifgalih-k/miniconda3/envs/myenv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Fine tuning VITS VCTK

In [4]:
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

In [5]:
def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [6]:
# hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")

In [7]:
# net_g_ms = SynthesizerTrn(
#     len(symbols),
#     hps_ms.data.filter_length // 2 + 1,
#     hps_ms.train.segment_size // hps_ms.data.hop_length,
#     n_speakers=hps_ms.data.n_speakers,
#     **hps_ms.model)
# _ = net_g_ms.eval()

# _ = utils.load_checkpoint("./logs/als_finetune_v5/G_1000.pth", net_g_ms, None)

In [8]:
# sid = torch.LongTensor([1]) # speaker identity
# stn_tst = get_text("We propose VITS, Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech.", hps_ms)

# with torch.no_grad():
#     x_tst = stn_tst.unsqueeze(0)
#     x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
#     audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
# ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate))

## Training FT

In [ ]:
!python train_ms_alt.py -c configs/vctk_base_alt.json -m als_ft_embedspeaker

INFO:als_ft_embedspeaker:{'train': {'log_interval': 50, 'eval_interval': 200, 'seed': 1234, 'epochs': 300, 'learning_rate': 0.0001, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 16, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 8192, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0, 'c_speaker_consistency': 0.1}, 'data': {'training_files': '/home/is/lathifgalih-k/research_naist/multimodal-clone/vits/filelists/als_train.txt.cleaned', 'validation_files': '/home/is/lathifgalih-k/research_naist/multimodal-clone/vits/filelists/als_val.txt.cleaned', 'text_cleaners': ['english_cleaners2'], 'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 9, 'cleaned_text': True}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resbl